In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_url = 'https://raw.githubusercontent.com/aso-uts/applied_ds/master/unit3/dataset/Car%20Evaluation.csv'

In [3]:
df = pd.read_csv(file_url)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   buying_price      1728 non-null   object
 1   maintenance_cost  1728 non-null   object
 2   doors             1728 non-null   object
 3   persons_capacity  1728 non-null   object
 4   luggage_boot      1728 non-null   object
 5   safety            1728 non-null   object
 6   evaluation        1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [4]:
df.to_csv('../data/raw/car_evaluation.csv', index=False)

In [28]:
df_cleaned = df.copy()

In [29]:
cats_dict = {
    'buying_price': [['low', 'med', 'high', 'vhigh']],
    'maintenance_cost': [['low', 'med', 'high', 'vhigh']],
    'doors': [['2', '3', '4', '5more']],
    'persons_capacity': [['2', '4', 'more']],
    'luggage_boot': [['small', 'med', 'big']],
    'safety': [['low', 'med', 'high']],
    'evaluation': [['unacc', 'acc', 'good', 'vgood']],
}

In [30]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [31]:
for col, cats in cats_dict.items():
    col_encoder = OrdinalEncoder(categories=cats)
    df_cleaned[col] = col_encoder.fit_transform(df_cleaned[[col]])

In [32]:
num_cols = ['buying_price', 'maintenance_cost', 'doors', 'persons_capacity', 'luggage_boot', 'safety']

In [33]:
sc = StandardScaler()

In [34]:
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [35]:
df_cleaned['evaluation'] = df_cleaned['evaluation'].astype(int)

In [36]:
X = pd.DataFrame(df_cleaned.copy())
df_cleaned.tail()

,buying_price,maintenance_cost,doors,persons_capacity,luggage_boot,safety,evaluation
1723,-1.341641,-1.341641,1.341641,1.224745,0.000000,0.000000,2
1724,-1.341641,-1.341641,1.341641,1.224745,0.000000,1.224745,3
1725,-1.341641,-1.341641,1.341641,1.224745,1.224745,-1.224745,0
1726,-1.341641,-1.341641,1.341641,1.224745,1.224745,0.000000,2
1727,-1.341641,-1.341641,1.341641,1.224745,1.224745,1.224745,3


In [37]:
from sklearn.model_selection import train_test_split

train_data_target = X.pop('evaluation')

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(X, train_data_target, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
from torch.utils.data import Dataset, DataLoader

class PytorchDataset(Dataset):
    """
    Pytorch dataset
    ...

    Attributes
    ----------
    X_tensor : Pytorch tensor
        Features tensor
    y_tensor : Pytorch tensor
        Target tensor

    Methods
    -------
    __getitem__(index)
        Return features and target for a given index
    __len__
        Return the number of observations
    to_tensor(data)
        Convert Pandas Series to Pytorch tensor
    """
        
    def __init__(self, X, y):
        self.X_tensor = self.to_tensor(X)
        self.y_tensor = self.to_tensor(y)
    
    def __getitem__(self, index):
        return self.X_tensor[index], self.y_tensor[index]
        
    def __len__ (self):
        return len(self.X_tensor)
    
    def to_tensor(self, data):
        return torch.Tensor(np.array(data))

In [42]:
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [43]:
import pandas as pd
import numpy as np

class NullModel:
    """
    Class used as baseline model for both regression and classification
    ...

    Attributes
    ----------
    target_type : str
        Type of ML problem (default regression)
    y : Numpy Array-like
        Target variable
    pred_value : Float
        Value to be used for prediction
    preds : Numpy Array
        Predicted array

    Methods
    -------
    fit(y)
        Store the input target variable and calculate the predicted value to be used based on the problem type
    predict(y)
        Generate the predictions
    fit_predict(y)
        Perform a fit followed by predict
    """
        
    
    def __init__(self, target_type: str = "regression"):
        self.target_type = target_type
        self.y = None
        self.pred_value = None
        self.preds = None
        
    def fit(self, y):
        self.y = y
        if self.target_type == "regression":
            self.pred_value = y.mean()
        else:
            from scipy.stats import mode
            self.pred_value = mode(y)[0][0]
    
    def predict(self, y):
        self.preds = np.full((len(y), 1), self.pred_value)
        return self.preds
    
    def fit_predict(self, y):
        self.fit(y)
        return self.predict(self.y)

In [44]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [45]:
def print_class_perf(y_preds, y_actuals, set_name=None, average='binary'):
    """Print the Accuracy and F1 score for the provided data

    Parameters
    ----------
    y_preds : Numpy Array
        Predicted target
    y_actuals : Numpy Array
        Actual target
    set_name : str
        Name of the set to be printed
    average : str
        Parameter  for F1-score averaging
    Returns
    -------
    """
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score

    print(f"Accuracy {set_name}: {accuracy_score(y_actuals, y_preds)}")
    print(f"F1 {set_name}: {f1_score(y_actuals, y_preds, average=average)}")

In [46]:
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.7005789909015715
F1 Training: 0.5772280207136489


In [47]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 32)
        self.layer_out = nn.Linear(32, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)

In [48]:
model = PytorchMultiClass(X_train.shape[1])

In [49]:
### awkward fix
device = torch.device('cpu')

In [50]:
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [51]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [52]:
criterion = nn.CrossEntropyLoss()

In [53]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [54]:
def train_classification(train_data, model, criterion, optimizer, batch_size, device, scheduler=None, generate_batch=None):
    """Train a Pytorch multi-class classification model

    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    optimizer: torch.optim
        Optimizer
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    scheduler : torch.optim.lr_scheduler
        Pytorch Scheduler used for updating learning rate
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """
    
    # Set model to training mode
    model.train()
    train_loss = 0
    train_acc = 0
    
    # Create data loader
    data = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:

        # Reset gradients
        optimizer.zero_grad()
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Make predictions
        output = model(feature)
        
        # Calculate loss for given batch
        loss = criterion(output, target_class.long())

        # Calculate global loss
        train_loss += loss.item()
        
        # Calculate gradients
        loss.backward()

        # Update Weights
        optimizer.step()
        
        # Calculate global accuracy
        train_acc += (output.argmax(1) == target_class).sum().item()

    # Adjust the learning rate
    if scheduler:
        scheduler.step()

    return train_loss / len(train_data), train_acc / len(train_data)

In [55]:
def test_classification(test_data, model, criterion, batch_size, device, generate_batch=None):
    """Calculate performance of a Pytorch multi-class classification model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0
    test_acc = 0
    
    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)
            
            # Calculate loss for given batch
            loss = criterion(output, target_class.long())

            # Calculate global loss
            test_loss += loss.item()
            
            # Calculate global accuracy
            test_acc += (output.argmax(1) == target_class).sum().item()

    return test_loss / len(test_data), test_acc / len(test_data)

In [56]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [57]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Epoch: 0
	(train)	|	Loss: 0.0316	|	Acc: 73.7%
	(valid)	|	Loss: 0.0319	|	Acc: 79.9%
Epoch: 1
	(train)	|	Loss: 0.0300	|	Acc: 79.0%
	(valid)	|	Loss: 0.0319	|	Acc: 80.3%
Epoch: 2
	(train)	|	Loss: 0.0294	|	Acc: 80.5%
	(valid)	|	Loss: 0.0314	|	Acc: 82.2%
Epoch: 3
	(train)	|	Loss: 0.0292	|	Acc: 81.5%
	(valid)	|	Loss: 0.0310	|	Acc: 83.4%
Epoch: 4
	(train)	|	Loss: 0.0287	|	Acc: 83.0%
	(valid)	|	Loss: 0.0310	|	Acc: 83.8%
Epoch: 5
	(train)	|	Loss: 0.0293	|	Acc: 81.0%
	(valid)	|	Loss: 0.0303	|	Acc: 86.1%
Epoch: 6
	(train)	|	Loss: 0.0290	|	Acc: 81.8%
	(valid)	|	Loss: 0.0316	|	Acc: 81.5%
Epoch: 7
	(train)	|	Loss: 0.0289	|	Acc: 82.4%
	(valid)	|	Loss: 0.0311	|	Acc: 83.0%
Epoch: 8
	(train)	|	Loss: 0.0290	|	Acc: 82.1%
	(valid)	|	Loss: 0.0306	|	Acc: 84.9%
Epoch: 9
	(train)	|	Loss: 0.0284	|	Acc: 83.9%
	(valid)	|	Loss: 0.0304	|	Acc: 85.3%
Epoch: 10
	(train)	|	Loss: 0.0288	|	Acc: 82.5%
	(valid)	|	Loss: 0.0305	|	Acc: 85.3%
Epoch: 11
	(train)	|	Loss: 0.0288	|	Acc: 82.7%
	(valid)	|	Loss: 0.0310	|	Acc: 83.4%
Ep

In [58]:
torch.save(model, "../models/pytorch_multi_car_evaluation.pt")

In [59]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0307	|	Accuracy: 0.8
